In [ ]:
!pip -q install pandas openpyxl numpy

In [ ]:
import pandas as pd
import numpy as np
from io import BytesIO

def to_clean_str(x):
    """Converteer waarde naar nette string, NaN -> lege string."""
    if pd.isna(x):
        return ""
    return str(x).strip()

def parse_int_safe(s):
    """Probeer een string veilig naar int te parsen, anders None."""
    try:
        if s is None:
            return None
        s = str(s).strip()
        if s == "":
            return None
        # staat toe dat Excel getallen als "2,0" of "2.0" levert
        return int(float(s.replace(",", ".")))
    except Exception:
        return None

# AANGEPAST: alleen de sheet "INVOER" inlezen
def load_all_sheets(filebytes: bytes) -> pd.DataFrame:
    """
    Lees alleen het tabblad 'INVOER' uit het Excel-bestand.
    Zo voorkomen we dat FORMULE of toekomstige tabs ruis geven.
    """
    xls = pd.ExcelFile(BytesIO(filebytes))
    frames = []
    for sheet in xls.sheet_names:
        if sheet != "INVOER":
            continue
        df = pd.read_excel(BytesIO(filebytes), sheet_name=sheet, header=0)
        df["__sheet__"] = sheet
        frames.append(df)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def find_scorers_column(df: pd.DataFrame) -> pd.Series:
    """
    Zoek kolom met doelpuntenmakers.
    Eerst op naam, anders heuristiek: kolom na de eerste ~10 kolommen
    met de meeste niet-numerieke waarden.
    """
    candidates = [
        c for c in df.columns
        if isinstance(c, str)
        and any(k in c.lower() for k in ["doelpunt", "makers", "scorer"])
    ]
    if candidates:
        return df[candidates[0]].apply(to_clean_str)

    best_i, best_score = None, -1
    for i, c in enumerate(df.columns):
        if i <= 10:
            continue
        s = df[c]
        cnt = 0
        for val in s.dropna().astype(str).values[:500]:
            try:
                float(val.replace(",", "."))
            except Exception:
                cnt += 1
        if cnt > best_score:
            best_score, best_i = cnt, i

    if best_i is not None:
        return df.iloc[:, best_i].apply(to_clean_str)
    else:
        return pd.Series([""] * len(df))

# AANGEPAST: 'beker' telt nu ook als divisie/klasse
def looks_like_division(text: str) -> bool:
    """
    Herken regel als een divisie-/klassekop.
    Nu ook 'beker' zodat het BEKER-blok een eigen subhead krijgt.
    """
    t = str(text or "").strip().lower()
    return (
        ("divisie" in t)
        or ("klasse" in t)
        or ("beker" in t)
    )

def build_output(filebytes: bytes) -> str:
    """
    Bouw de tekstoutput (<body>...</body>) uit het Excelbestand.
    Verwacht het INVOER-tabblad in AMA_uitslagen_amateuvoetbal_20xx-20xx.xlsx.
    """
    raw = load_all_sheets(filebytes)
    if raw.empty:
        raise RuntimeError("Geen data gevonden in het Excelbestand.")

    def get_col(df, idx, fallback):
        # veilige kolomselectie op index
        if df.shape[1] > idx:
            return df.iloc[:, idx].apply(to_clean_str)
        return pd.Series([""] * len(df), name=fallback)

    # vaste kolomposities in INVOER
    home = get_col(raw, 1, "Thuisclub")    # kolom B
    away = get_col(raw, 3, "Uitclub")      # kolom D
    hg   = get_col(raw, 5, "ThuisGoals")   # kolom F
    ag   = get_col(raw, 7, "UitGoals")     # kolom H
    hht  = get_col(raw, 8, "RustThuis")    # kolom I
    aht  = get_col(raw, 10, "RustUit")     # kolom K
    scor = find_scorers_column(raw)

    lines = ["<body>"]

    # Init: probeer kop van de tweede kolom als eerste divisie te herkennen
    second_col_header = str(raw.columns[1]) if len(raw.columns) > 1 else ""
    current_div = second_col_header.upper() if looks_like_division(second_col_header) else None
    emitted_div = False

    n = len(raw)
    for i in range(n):
        home_cell = home.iloc[i]
        away_cell = away.iloc[i]
        hg_raw = hg.iloc[i]
        ag_raw = ag.iloc[i]
        hht_raw = hht.iloc[i]
        aht_raw = aht.iloc[i]
        scorers = scor.iloc[i] if i < len(scor) else ""

        # als een cel een nieuwe divisie/klasse/Beker-kop is
        if looks_like_division(home_cell):
            current_div = home_cell.upper()
            emitted_div = False
            continue

        # lege regels overslaan (geen thuis- of uitclub)
        if not (home_cell and home_cell.strip()) or not (away_cell and away_cell.strip()):
            continue

        # één keer per blok een subhead_lead met de divisienaam
        if current_div and not emitted_div:
            lines.append(f"<subhead_lead>{current_div}</subhead_lead>")
            emitted_div = True

        # uitgestelde wedstrijd?
        hg_text = to_clean_str(hg_raw)
        postponed = ("afg" in hg_text.lower()) or ("gest" in hg_text.lower())
        hg_num = parse_int_safe(hg_raw)
        ag_num = parse_int_safe(ag_raw)

        # geen doelpuntenmakers tonen bij 0–0 (tenzij afg./gest.)
        if not postponed and hg_num == 0 and ag_num == 0:
            scorers = " "

        if postponed:
            # bv. "afg." of "gest." in de eindstand
            subhead = f"<subhead>{home_cell} - {away_cell} {hg_text}</subhead>"
        else:
            tg = 0 if hg_num is None else int(hg_num)
            ug = 0 if ag_num is None else int(ag_num)
            rth = 0 if parse_int_safe(hht_raw) is None else int(parse_int_safe(hht_raw))
            rut = 0 if parse_int_safe(aht_raw) is None else int(parse_int_safe(aht_raw))
            # scores zonder spaties rond streepje
            subhead = f"<subhead>{home_cell} - {away_cell} {tg}-{ug} ({rth}-{rut})</subhead>"

        lines.append(subhead)
        lines.append("<howto_facts>")
        lines.append(scorers)
        lines.append("</howto_facts>")

    lines.append("</body>")
    return "\n".join(lines)


In [ ]:
from google.colab import files

uploaded = files.upload()  # kies hier het Excel-bestand (INVOER/FORMULE)
key = next(iter(uploaded.keys()))
print('Bestand:', key)

txt = build_output(uploaded[key])
with open('uitslagen_output.txt', 'w', encoding='utf-8') as f:
    f.write(txt)

print('TXT aangemaakt: uitslagen_output.txt')

In [ ]:
with open('uitslagen_output.txt', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        print(line.rstrip())
        if i > 80:
            print('... (ingekort) ...')
            break

In [ ]:
from google.colab import files
files.download('uitslagen_output.txt')